## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import os

from tqdm import tqdm


## Datasets Preparation

In [ ]:
def datasets_preparation():

    ds_name = ('/results2/SalishSea/nowcast-green.202111/' + i + '/SalishSea_1d_' + '20' + str(i[5:7]) + str(dict_month[i[2:5]])+str(i[0:2]) + '_' + '20' + str(i[5:7]) + str(dict_month[i[2:5]]) + str(i[0:2]) + '_grid_T.nc')

    ds_bio_name = ('/results2/SalishSea/nowcast-green.202111/' + i + '/SalishSea_1d_'  + '20' + str(i[5:7]) + str(dict_month[i[2:5]])+str(i[0:2]) + '_' + '20' + str(i[5:7]) + str(dict_month[i[2:5]]) + str(i[0:2]) + '_biol_T.nc')

    ds_prod_name = ('/results2/SalishSea/nowcast-green.202111/' + i + '/SalishSea_1d_'  + '20' + str(i[5:7]) + str(dict_month[i[2:5]])+str(i[0:2]) + '_' + '20' + str(i[5:7]) + str(dict_month[i[2:5]]) + str(i[0:2]) + '_prod_T.nc')

    ds = xr.open_dataset (ds_name)
    ds_bio = xr.open_dataset (ds_bio_name)
    ds_prod = xr.open_dataset(ds_prod_name)

    temp_i1 = ((ds.votemper.where(mask==1)[0,0:15] * ds.e3t.where(mask==1)
        [0,0:15]).sum('deptht', skipna = True, min_count = 15) / mesh.gdepw_0[0,15])

    temp_i2 = ((ds.votemper.where(mask==1)[0,15:27] * ds.e3t.where(mask==1)
        [0,15:27]).sum('deptht', skipna = True, min_count = 12) / (mesh.gdepw_0
        [0,27] - mesh.gdepw_0[0,14]))

    saline_i1 = ((ds.vosaline.where(mask==1)[0,0:15] * ds.e3t.where(mask==1)
        [0,0:15]).sum('deptht', skipna = True, min_count = 15) / mesh.gdepw_0[0,15])

    saline_i2 = ((ds.vosaline.where(mask==1)[0,15:27] * ds.e3t.where(mask==1)
        [0,15:27]).sum('deptht', skipna = True, min_count = 12) / (mesh.gdepw_0
        [0,27] - mesh.gdepw_0[0,14]))

    sil_i = ((ds_bio.silicon.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    nitr_i1 = (ds_bio.nitrate.where(mask==1)[0,0:15] * ds.e3t.where(mask==1)
        [0,0:15]).sum('deptht', skipna = True, min_count = 15) / mesh.gdepw_0[0,15]
    nitr_i2 = (ds_bio.nitrate.where(mask==1)[0,15:27] * ds.e3t.where(mask==1)
        [0,15:27]).sum('deptht', skipna = True, min_count = 12) / (mesh.gdepw_0[0,27] - mesh.gdepw_0[0,14])

    ammo_i = ((ds_bio.ammonium.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    diat_i = ((ds_bio.diatoms.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    flag_i = ((ds_bio.flagellates.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    micro_i = ((ds_bio.microzooplankton.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    meso_i = ((ds_bio.mesozooplankton.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])

    diat_i_prod = ((ds_prod.PPDIAT.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / diat_i

    return (temp_i1, temp_i2, saline_i1, saline_i2, sil_i, nitr_i1, nitr_i2, ammo_i, diat_i, flag_i, micro_i, meso_i, diat_i_prod)


## File Creation

In [ ]:
def file_creation(variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path='/data/ibougoudis/MOAD/analysis-ilias/notebooks/integrated.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})


## Data Collection

In [ ]:
dict_month = {'jan': '01',
    'feb': '02',
    'mar': '03',
    'apr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'aug': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dec': '12'}

path = os.listdir('/results2/SalishSea/nowcast-green.202111/')

folders = [x for x in path if ((x[2:5]=='mar' or x[2:5]=='apr' or (x[2:5]=='feb' and x[0:2] > '14')) and (x[5:7]<'24'))]
indx_dates=(np.argsort(pd.to_datetime(folders, format="%d%b%y")))
folders = [folders[i] for i in indx_dates]

# Open the mesh mask
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
mask = mesh.tmask.to_numpy()

for i in tqdm(folders):

    temp_i1, temp_i2, saline_i1, saline_i2, sil_i, nitr_i1, nitr_i2, ammo_i, diat_i, flag_i, micro_i, meso_i, diat_i_prod = datasets_preparation()

    if i==folders[0]:

        temp_i1_all = temp_i1
        temp_i2_all = temp_i2
        saline_i1_all = saline_i1
        saline_i2_all = saline_i2

        sil_i_all = sil_i
        nitr_i1_all = nitr_i1
        nitr_i2_all = nitr_i2
        ammo_i_all = ammo_i

        diat_i_all = diat_i
        flag_i_all = flag_i

        micro_i_all = micro_i
        meso_i_all = meso_i

    else:

        temp_i1_all = xr.concat((temp_i1_all, temp_i1), dim='time_counter')
        temp_i2_all = xr.concat((temp_i2_all, temp_i2), dim='time_counter')
        saline_i1_all = xr.concat((saline_i1_all, saline_i1), dim='time_counter')
        saline_i2_all = xr.concat((saline_i2_all, saline_i2), dim='time_counter')

        sil_i_all = xr.concat((sil_i_all, sil_i), dim='time_counter')
        nitr_i1_all =  xr.concat((nitr_i1_all, nitr_i1), dim='time_counter')
        nitr_i2_all =  xr.concat((nitr_i2_all, nitr_i2), dim='time_counter')
        ammo_i_all = xr.concat((ammo_i_all, ammo_i), dim='time_counter')

        diat_i_all = xr.concat((diat_i_all, diat_i), dim='time_counter')
        flag_i_all = xr.concat((flag_i_all, flag_i), dim='time_counter')

        micro_i_all = xr.concat((micro_i_all, micro_i), dim='time_counter')
        meso_i_all = xr.concat((meso_i_all, meso_i), dim='time_counter')

# Calling file creation

file_creation(temp_i1_all, 'Temperature (0m-15m)')
file_creation(temp_i2_all, 'Temperature (15m-100m)')
file_creation(saline_i1_all, 'Salinity (0m-15m)')
file_creation(saline_i2_all, 'Salinity (15m-100m)')

file_creation(sil_i_all, 'Silicon')
file_creation(nitr_i1_all, 'Nitrate (0m-15m)')
file_creation(nitr_i2_all, 'Nitrate (15m-100m)')
file_creation(ammo_i_all, 'Ammonium')

file_creation(diat_i_all, 'Diatom')
file_creation(flag_i_all, 'Flagellate')

file_creation(micro_i_all, 'Microzooplankton')
file_creation(meso_i_all, 'Mesozooplankton')
